# Setups

In [5]:
# Imports
import numpy as np
import time
import matplotlib.pyplot as plt
# Packages required for EMRI waveforms
from few.waveform import GenerateEMRIWaveform

import h5py
from ldc.lisa.orbits import OrbitsFromFile
from ldc.lisa.projection import ProjectedStrain

from ldc.common.tools import window

from ldc.common.series import TDI, XYZ2AET
from ldc.lisa.noise import get_noise_model
from ldc.waveform.waveform import NumericHpHc


from tqdm import tqdm


# Helping functions

def inner_product(lhsA, lhsE, rhsA, rhsE, SA, df):
    return 4.0 * df * np.sum(np.real(lhsA * np.conj(rhsA) + lhsE * np.conj(rhsE)) / SA)


def fourier(data, dt, n=0):
    """
    params: data - list like with elements - arrays-like of equal size
    return: list - fourier transforms, frequencies
    """
    if n == 0:
        n = data[0].size

    for i in range(len(data)):
        data[i] = np.fft.rfft(data[i], n)[1:]

    freq = np.fft.rfftfreq(n, d=dt)[1:]  # cause we want freq[0] != 0
    return data, freq


def crop_data(data, freq, fmin, fmax):
    """
    params: data - list like with elements - arrays-like of equal size; freq - array-like of original frequencies;
    return: list - cropped data, cropped frequencies
    """
    if fmin == 0 and fmax == np.inf:
        return data, freq

    n = freq.size
    imin, imax = 0, n - 1
    for i in range(n):
        if freq[i] > fmin:
            imin = i
            break
    for i in range(n - 1, -1, -1):
        if freq[i] < fmax:
            imax = i
            break
    for i in range(len(data)):
        data[i] = data[i][imin:imax]
    freq = freq[imin:imax]
    return data, freq


class LikelihoodCalculator:

    def __init__(self, Phi_phi0, Phi_theta0, Phi_r0, T, dt, priors):
        self.Phi_phi0 = Phi_phi0
        self.Phi_theta0 = Phi_theta0
        self.Phi_r0 = Phi_r0
        self.T = T  # in years
        self.dt = dt  # in seconds
        self.orbit_file = 'equalarmlength-orbits.h5'
        self.priors = priors

        with h5py.File(self.orbit_file) as f:
            L = f.attrs['L']

        self.orbits = OrbitsFromFile({
            'orbit_type': 'file',
            'filename': self.orbit_file,
            'nominal_arm_length': L},
            read_t0=False,
        )

        self.dA = None  # in freq. domain
        self.dE = None  # in freq. domain
        self.freq = None
        # self.addition = None  # equal to -1/2 (d, d)
        self.SA = None
        self.n = None  # size of original signal in time
        self.df = None

    def get_tdi(self, M, mu, a, p0, e0, x0, dist, qS, phiS, qK, phiK):
        # Generating EMRI waveform
        gen_wave = GenerateEMRIWaveform("Pn5AAKWaveform", use_gpu=False)
        h_aak = gen_wave(
            M,
            mu,
            a,
            p0,
            e0,
            x0,
            dist,
            qS,
            phiS,
            qK,
            phiK,
            self.Phi_phi0,
            self.Phi_theta0,
            self.Phi_r0,
            T=self.T,
            dt=self.dt,
        )

        end_index, = h_aak.shape
        t_merge = end_index * self.dt

        # Calculating response
        t_min = 0
        t_max = t_merge + 1000
        t = np.arange(t_min, t_max, self.dt)
        h_aak = np.concatenate([h_aak, np.zeros(t.size - h_aak.size)])

        hphc_num = NumericHpHc(t, h_aak.real, h_aak.imag, qS, phiS)
        projected_strain = ProjectedStrain(self.orbits)
        projected_strain.arm_response(t_min + 500, t_max - 500, self.dt, [hphc_num])

        # Calculating TDI
        X_ldc = projected_strain.compute_tdi_x(t)
        Y_ldc = projected_strain.compute_tdi_y(t)
        Z_ldc = projected_strain.compute_tdi_z(t)

        # Applying window function
        window_function = window(t)
        X_ldc *= window_function
        Y_ldc *= window_function
        Z_ldc *= window_function

        # Transform to A, E
        A, E, _ = XYZ2AET(X_ldc, Y_ldc, Z_ldc)

        return A, E

    def setup(self, M, mu, a, p0, e0, x0, dist, qS, phiS, qK, phiK):
        A, E = self.get_tdi(M, mu, a, p0, e0, x0, dist, qS, phiS, qK, phiK)

        self.n = A.size
        [self.dA, self.dE], self.freq = fourier([A, E], self.dt)
        [self.dA, self.dE], self.freq = crop_data([self.dA, self.dE], self.freq, 6e-4, 2e-2)

        self.df = self.freq[1] - self.freq[0]

        # Setup noise model
        noise = get_noise_model("SciRDv1", self.freq, wd=1)
        self.SA = noise.psd(self.freq)
        # self.addition = - 0.5 * inner_product(self.dA, self.dE, self.dA, self.dE, self.SA, self.df)

        SN2 = 4.0 * self.df * np.sum((np.abs(self.dA) ** 2 + np.abs(self.dE) ** 2) / self.SA)
        print("Setup successful!")
        print("SNR of the original signal =", round(np.sqrt(SN2), 3))
        # plt.loglog(self.freq, np.abs(self.dA))
        # plt.loglog(self.freq, np.sqrt(self.SA))
        # plt.show()

    def setup_with_sangria(self, M, mu, a, p0, e0, x0, dist, qS, phiS, qK, phiK):
        A, E = self.get_tdi(M, mu, a, p0, e0, x0, dist, qS, phiS, qK, phiK)

        sangria_fn = "LDC2_sangria_training_v2.h5"
        tdi_ts = TDI.load(sangria_fn, name="obs/tdi")
        tdi_mbhb_ts = TDI.load(sangria_fn, name="sky/mbhb/tdi")

        tdi_ts -= tdi_mbhb_ts
        tdi_ts.XYZ2AET()

        self.n = tdi_ts['A'].values.size
        A = np.concatenate([A, np.zeros(self.n - A.size)])
        E = np.concatenate([E, np.zeros(self.n - E.size)])
        tdi_ts['A'].values += A
        tdi_ts['E'].values += E

        [self.dA, self.dE], self.freq = fourier([tdi_ts['A'].values, tdi_ts['E'].values], self.dt)
        [self.dA, self.dE], self.freq = crop_data([self.dA, self.dE], self.freq, 5e-4, 2e-2)

        self.df = self.freq[1] - self.freq[0]

        # Setup noise model
        noise = get_noise_model("sangria", self.freq, wd=1)
        self.SA = noise.psd(self.freq)
        # self.addition = - 0.5 * inner_product(self.dA, self.dE, self.dA, self.dE, self.SA, self.df)

        # SN2 = 4.0 * self.df * np.sum((np.abs(self.dA) ** 2 + np.abs(self.dE) ** 2) / self.SA)
        print("Setup successful!")
        # print("SNR of the original signal =", round(np.sqrt(SN2), 3))
        # plt.loglog(self.freq, np.abs(self.dA), label="dA")
        # plt.loglog(self.freq, np.sqrt(self.SA), label="SA")
        # plt.xlabel("freq. [Hz]")
        # plt.legend()
        # plt.show()
    
    # Method just to this particular task (calculating derivatives)
    def get_h(self, point):
        qS = np.arccos(point[7])
        qK = np.arccos(point[9])
        try:
            A, E = self.get_tdi(point[0], point[1], point[2], point[3], point[4], point[5], point[6], qS, point[8], qK,
                                point[10])
        except ValueError:
            return None
        [hA, hE], freq = fourier([A, E], self.dt, self.n)
        return crop_data([hA, hE], freq, 5e-4, 2e-2)

In [6]:
import numpy as np
import mc3.proposal as proposal
from mc3.sampler import PTSampler
import mc3.gb as gbtools

# Initialize true parameters of the signal
T = 0.733  # observation time (years)
dt = 5.0  # time step (seconds)
M = 507236.8057121273 * 1.2  # large mass (solar)
a = 0.6  # spin / will be ignored in Schwarzschild waveform
mu = 18.063091389346287  # small mass (solar)
p0 = 10.752251937834985  # initial separation
e0 = 0.3857270761433499  # initial eccentricity
x0 = 0.7  # initial cosine of the inclination / will be ignored in Schwarzschild waveform
qK = np.pi / 2 - 0.6275167236796371  # polar spin angle
phiK = 1.7262549907689677  # azimuthal viewing angle
qS = 2.17583780178878  # polar sky angle
phiS = 0.7101021513597163  # azimuthal viewing angle
dist = 1.4357198958825074 / 2  # distance
Phi_phi0 = 0  # initial phase in phi
Phi_theta0 = 1.7262549907689677  # initial phase in theta
Phi_r0 = 0  # initial phase in r

In [7]:
priors = [[M * 0.6, M * 1.4], [mu * 0.6, mu * 1.4], [0.1, 0.9], [10, 12], [0.05, 0.5], [-0.9, 0.9], [dist * 0.5, dist * 1.5], [-1.0, 1.0],
              [0.0, 2 * np.pi], [-1.0, 1.0], [0.0, 2 * np.pi]]

In [ ]:
calculator = LikelihoodCalculator(Phi_phi0, Phi_theta0, Phi_r0, T, dt, priors)
calculator.setup_with_sangria(M, mu, a, p0, e0, x0, dist, qS, phiS, qK, phiK)

In [ ]:
x0 = [M, mu, a, p0, e0, x0, dist, np.cos(qS), phiS, np.cos(qK), phiK] # Start (true) point
names = ["M", "mu", "spin", "p0", "e0", "x0", "dist", "cos(qS)", "phiS", "cos(qK)", "phiK"]
x0_numpy = np.array(x0)

# Calculating derivatives

$\langle \partial_i h, \partial_j h \rangle \approx \langle \dfrac{h(x_0 + dx_i) - h(x_0 - dx_i)}{2 |dx_i|} , \dfrac{h(x_0 + dx_j) - h(x_0 - dx_j)}{2 |dx_j|} \rangle $

In [6]:
def compute_term(calculator, names, x0, dx_i: float, i: int):
    dx = np.zeros_like(x0)
    
    if names[i] == "dist":
        [result_A, result_E], _ = calculator.get_h(x0)
        result_A /= -x0[i]
        result_E /= -x0[i]
        
    elif names[i] == "log M":
        dx[i] = dx_i
        [hA_i1, hE_i1], _ = calculator.get_h(x0 + dx)
        [hA_i2, hE_i2], _ = calculator.get_h(x0 - dx)
        
        result_A = hA_i1 - hA_i2
        result_E = hE_i1 - hE_i2
        
        result_A /= 2 * dx_i
        result_E /= 2 * dx_i
        
        result_A /= np.exp(x0[i])
        result_E /= np.exp(x0[i])
        
    else:
        dx[i] = dx_i
        [hA_i1, hE_i1], _ = calculator.get_h(x0 + dx)
        [hA_i2, hE_i2], _ = calculator.get_h(x0 - dx)
        
        result_A = hA_i1 - hA_i2
        result_E = hE_i1 - hE_i2
        
        result_A /= 2 * dx_i
        result_E /= 2 * dx_i
        
    return result_A, result_E



def calculating_gamma_matrix(calculator, names, x0, true_dx):
    
    terms = []    
    for i in tqdm(range(len(x0))):
        terms.append(compute_term(calculator, names, x0, true_dx[names[i]], i))
    
    df = calculator.df
    SA = calculator.SA
    
    A_matrix = np.zeros((len(x0), len(x0)))

    for i in range(len(x0)):
        for j in range(i + 1):
            A_matrix[i][j] = inner_product(terms[i][0], terms[i][1], terms[j][0], terms[j][1], SA, df)
            
            
            A_matrix[j][i] = np.conj(A_matrix[i][j])
            
    return A_matrix, terms

In [9]:
true_dx = {}
true_dx["M"] = 1e-4 * x0_numpy[0]
true_dx["mu"] = 1e-4 * x0_numpy[1]
true_dx["spin"] = 5e-4
true_dx["p0"] = 5e-4
true_dx["e0"] = 5e-4
true_dx["x0"] = 5e-4
true_dx["dist"] = 1e-5
true_dx["cos(qS)"] = 2e-4
true_dx["phiS"] = 1e-4
true_dx["cos(qK)"] = 4e-2 * x0_numpy[9]
true_dx["phiK"] = 5e-2 * x0_numpy[10]

In [ ]:
gamma_matrix, terms = calculating_gamma_matrix(calculator, names, x0_numpy, true_dx)

In [9]:
gamma_matrix = np.load("./save_files/Gamma_matrix.npy")

# Computing $\Delta \theta$

In [11]:
sangria_fn = "LDC2_sangria_training_v2.h5"
tdi_ts = TDI.load(sangria_fn, name="obs/tdi")
tdi_mbhb_ts = TDI.load(sangria_fn, name="sky/mbhb/tdi")

tdi_ts -= tdi_mbhb_ts
tdi_ts.XYZ2AET()

In [12]:
def compute_term_2(calculator, terms, tdi_ts):
    
    df = calculator.df
    SA = calculator.SA
    
    [dA, dE], freq = fourier([tdi_ts['A'].values, tdi_ts['E'].values], calculator.dt)
    [dA, dE], freq = crop_data([dA, dE], freq, 5e-4, 2e-2)
    
    term_2 = []
    
    for term in tqdm(terms):
        term_2.append(inner_product(term[0], term[1], dA, dE, SA, df))
        
    return term_2

In [ ]:
term_2 = compute_term_2(calculator, terms, tdi_ts)

In [10]:
term_2 = np.load("./save_files/term_2.npy")

In [12]:
delta_theta = np.linalg.inv(gamma_matrix) @ term_2

In [13]:
delta_theta

array([-1.44254907e-01, -2.69333714e-05,  3.66870682e-05,  1.83333006e-05,
       -2.70236511e-05,  2.64196934e-07,  1.68221243e-02, -2.10766235e-03,
       -1.36294194e-03, -1.34155732e-02, -1.00913169e-02])